In [19]:
import csv
import random
import torch

from reasoner import Reasoner
from world import World

from collections import defaultdict
from tqdm import tqdm

In [2]:
random.seed(42)
torch.manual_seed(42)

model_lr = {
    'rl-property': 2.5074850299401197e-06,
    'bl-property': 2.508483033932136e-06,
    'axxl-property': 3.0054890219560877e-06,
}

negative_sampler = {
    'axxl-property': '../../induction/checkpoints/finetuned_models/axxl-property',
    'bl-property': '../../induction/checkpoints/finetuned_models/bl-property',
    'rl-property': '../../induction/checkpoints/finetuned_models/rl-property',
}

world = World(concept_path = "../data/concept_senses.csv", 
             feature_path = '../data/experimental splits/train_1ns.csv', 
             matrix_path = "../data/train_1ns_matrix.txt")
world.create()

521it [00:00, 6725.80it/s]


In [3]:
PROPERTIES = ['can dax', 'can fep', 'is vorpal', 'is mimsy', 'has blickets', 'has feps', 'is a wug', 'is a tove']

In [4]:
MODELS = ['axxl-property', 'bl-property', 'rl-property']
DEVICE = 'cuda:0'

In [5]:
categories = ['bird.n.01', 'mammal.n.01', 'reptile.n.01', 'fish.n.01', 'insect.n.01', 'mollusk.n.01']

candidates = []
for cat in categories:
    descendants = world.taxonomy[cat].descendants()
    for concept in descendants:
        negatives = set(categories) - set([cat])
        positive = [c for c in world.taxonomy[cat].descendants() if c != concept]
        for cn in negatives:
            negative = [c for c in world.taxonomy[cn].descendants()]

            sim_p = world.similarity([concept], positive).mean().item()
            sim_n = world.similarity([concept], negative).mean().item()

            if sim_n > sim_p:
                candidates.append((concept, cat, cn))

In [6]:
candidates

[('hippo', 'mammal.n.01', 'reptile.n.01'),
 ('dolphin', 'mammal.n.01', 'fish.n.01'),
 ('whale', 'mammal.n.01', 'fish.n.01'),
 ('turtle', 'reptile.n.01', 'fish.n.01'),
 ('slug', 'mollusk.n.01', 'insect.n.01'),
 ('snail', 'mollusk.n.01', 'insect.n.01')]

In [7]:
random.seed(42)

stimuli = []
for concept, p, n in candidates:
    negative = world.taxonomy[n].descendants()
    positive = [c for c in world.taxonomy[p].descendants() if c != concept]
#     positive = random.sample(positive, min(len(positive), len(negative)))
    
    stimuli.append([concept, positive, negative])

In [8]:
concept = 'oyster'
negative = 'mollusk'

world.similarity([concept], [c for c in world.taxonomy[f'{negative}.n.01'].descendants() if c != concept]).mean()

tensor(0.4069)

In [9]:
# precompute random samples

fish = [c for c in world.taxonomy['fish.n.01'].descendants()]

neg_samples = {
    'dolphin': random.sample([c for c in world.taxonomy['mammal.n.01'].descendants() if c != 'dolphin'], len(fish)),
    'whale': random.sample([c for c in world.taxonomy['mammal.n.01'].descendants() if c != 'whale'], len(fish))
}

In [10]:
results = defaultdict(dict)

for MODEL in MODELS:
    result = {'within': [], 'outside': []}
    for prop in tqdm(PROPERTIES):
        reasoner = Reasoner(f'../../induction/checkpoints/finetuned_models/{MODEL}',
                    learning_rate = model_lr[MODEL], 
                    lexicon = world.lexicon,
                    device = DEVICE)
        
        for stimulus in stimuli:
            concept, within, outside = stimulus
            
            within_stimuli = reasoner.prepare_stimuli(within, prop)
            outside_stimuli = reasoner.prepare_stimuli(outside, prop)
            adaptation = reasoner.prepare_stimuli([concept], prop)
            
            reasoner.adapt(adaptation, labels = torch.tensor([1]))
            
            generalization_within = reasoner.generalize(within_stimuli)[:,1].mean().item()
            generalization_outside = reasoner.generalize(outside_stimuli)[:,1].mean().item()
            
            print('================')
            print(f'Property: {prop}, Concept: {concept}')
            print(f'WITHIN: {generalization_within}')
            print(f'OUTSIDE: {generalization_outside}')
            print('================')
            
            result['within'].append(generalization_within)
            result['outside'].append(generalization_outside)
            
    results[MODEL] = result
            

  0%|                                                           | 0/8 [00:00<?, ?it/s]

Property: can dax, Concept: hippo
WITHIN: -0.07642970234155655
OUTSIDE: -0.0909963846206665
Property: can dax, Concept: dolphin
WITHIN: -0.03356577455997467
OUTSIDE: -0.2745608985424042
Property: can dax, Concept: whale
WITHIN: -0.0193018801510334
OUTSIDE: -0.15030813217163086
Property: can dax, Concept: turtle
WITHIN: -0.014728267677128315
OUTSIDE: -0.0884445533156395


 12%|██████▍                                            | 1/8 [00:04<00:29,  4.28s/it]

Property: can dax, Concept: slug
WITHIN: -0.046209368854761124
OUTSIDE: -0.014397714287042618
Property: can dax, Concept: snail
WITHIN: -0.018888643011450768
OUTSIDE: -0.012600679881870747
Property: can fep, Concept: hippo
WITHIN: -0.35351529717445374
OUTSIDE: -0.3987973630428314
Property: can fep, Concept: dolphin
WITHIN: -0.15818661451339722
OUTSIDE: -0.5636946558952332
Property: can fep, Concept: whale
WITHIN: -0.07265093177556992
OUTSIDE: -0.3153703510761261
Property: can fep, Concept: turtle
WITHIN: -0.04054848477244377
OUTSIDE: -0.16635607182979584


 25%|████████████▊                                      | 2/8 [00:06<00:18,  3.09s/it]

...

Property: has feps, Concept: snail
WITHIN: -0.19223523139953613
OUTSIDE: -0.17207512259483337
Property: is a wug, Concept: hippo
WITHIN: -0.3737499713897705
OUTSIDE: -0.41341498494148254
Property: is a wug, Concept: dolphin
WITHIN: -0.1943567991256714
OUTSIDE: -0.8329520225524902
Property: is a wug, Concept: whale
WITHIN: -0.09902273118495941
OUTSIDE: -0.433041512966156
Property: is a wug, Concept: turtle
WITHIN: -0.060935813933610916
OUTSIDE: -0.32584595680236816


 88%|████████████████████████████████████████████▋      | 7/8 [00:16<00:02,  2.36s/it]

Property: is a wug, Concept: slug
WITHIN: -0.33513206243515015
OUTSIDE: -0.3780505955219269
Property: is a wug, Concept: snail
WITHIN: -0.2272053211927414
OUTSIDE: -0.2800273895263672
Property: is a tove, Concept: hippo
WITHIN: -0.6682340502738953
OUTSIDE: -0.5302767157554626
Property: is a tove, Concept: dolphin
WITHIN: -0.14324714243412018
OUTSIDE: -0.43861860036849976
Property: is a tove, Concept: whale
WITHIN: -0.0910964086651802
OUTSIDE: -0.32468798756599426


100%|███████████████████████████████████████████████████| 8/8 [00:18<00:00,  2.33s/it]

Property: is a tove, Concept: turtle
WITHIN: -0.04761747643351555
OUTSIDE: -0.2462242990732193
Property: is a tove, Concept: slug
WITHIN: -0.2682441771030426
OUTSIDE: -0.24820910394191742
Property: is a tove, Concept: snail
WITHIN: -0.1989985555410385
OUTSIDE: -0.18689486384391785


In [11]:
for k, v in results.items():
    print(f"MODEL: {k}")
    print(f"WITHIN: {torch.tensor(v['within']).mean()} (+= {torch.tensor(v['within']).std()})")
    print(f"OUTSIDE: {torch.tensor(v['outside']).mean()} (+= {torch.tensor(v['outside']).std()})")

MODEL: axxl-property
WITHIN: -0.17375223338603973 (+= 0.17410357296466827)
OUTSIDE: -0.28315630555152893 (+= 0.2764502465724945)
MODEL: bl-property
WITHIN: -0.187341570854187 (+= 0.16346880793571472)
OUTSIDE: -0.3000130355358124 (+= 0.19028933346271515)
MODEL: rl-property
WITHIN: -0.18275661766529083 (+= 0.128874734044075)
OUTSIDE: -0.33261069655418396 (+= 0.19451898336410522)


In [22]:
with open("../data/results/tease_apart.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(['model', 'within', 'outside'])
    for k,v in results.items():
        for w, o in zip(v['within'], v['outside']):
            writer.writerow([k, w, o])
#     print("=" * 80)
#     print(f"a <- c({', '.join([str(x) for x in v['within']])})")
#     print(f"b <- c({', '.join([str(x) for x in v['outside']])})")
#     print('t.test(a, b, paired = TRUE)')

In [5]:
PROPERTY = 'has feps'
CONCEPT = 'dolphin'

adaptation = reasoner.prepare_stimuli([CONCEPT], PROPERTY)
reasoner.adapt(adaptation, labels = torch.tensor([1]))


# birds = [c for c in world.taxonomy['bird.n.01'].descendants()]
mammals = [c for c in world.taxonomy['mammal.n.01'].descendants() if c != 'dolphin']
fish = [c for c in world.taxonomy['fish.n.01'].descendants() if c != 'dolphin']

mammal_stimuli = reasoner.prepare_stimuli(mammals, PROPERTY)
fish_stimuli = reasoner.prepare_stimuli(fish, PROPERTY)
# bird_stimuli = reasoner.prepare_stimuli(birds, PROPERTY)

In [6]:
reasoner.generalize(mammal_stimuli)[:,1].mean()

tensor(-0.2595)

In [7]:
reasoner.generalize(fish_stimuli)[:,1].mean()

tensor(-0.5563)

In [5]:
PROPERTY = 'has feps'

adaptation = reasoner.prepare_stimuli(["dolphin"], PROPERTY)
reasoner.adapt(adaptation, labels = torch.tensor([1]))


mammals = [c for c in world.taxonomy['mammal.n.01'].descendants() if c != 'dolphin']
fish = [c for c in world.taxonomy['fish.n.01'].descendants() if c != 'dolphin']

mammal_stimuli = reasoner.prepare_stimuli(mammals, PROPERTY)
fish_stimuli = reasoner.prepare_stimuli(fish, PROPERTY)

In [6]:
reasoner.generalize(mammal_stimuli)[:, 1].mean()

tensor(-0.1707)

In [7]:
reasoner.generalize(fish_stimuli)[:, 1].mean()

tensor(-0.6013)

In [15]:
world.similarity(['dolphin'], mammals).mean().item()

0.27070605754852295

In [16]:
world.similarity(['dolphin'], fish).mean().item()

0.30332738161087036

In [5]:
def prepare_stimuli_prop(self, concepts, properties):
    sentences = [f'{c} {p}.' for c in concepts for p in properties]
    print(sentences)
    encoded = self.tokenizer(sentences, return_tensors = 'pt', padding = True)
    
    return encoded

In [10]:
properties = ['is an animal', 'is a mammal', 'is an ape', 'can breathe', 'can grow', 'is a living thing', 'is a plant', 'is a fish']

stimuli = prepare_stimuli_prop(reasoner, ['a bird'], properties)

['a bird is an animal.', 'a bird is a mammal.', 'a bird is an ape.', 'a bird can breathe.', 'a bird can grow.', 'a bird is a living thing.', 'a bird is a plant.', 'a bird is a fish.']


In [11]:
reasoner.generalize(stimuli)[:,1].exp()

tensor([0.9936, 0.0117, 0.0059, 0.9947, 0.9946, 0.9874, 0.0046, 0.0063])

In [7]:
PROPERTY = "is a bird"
adaptation = prepare_stimuli_prop(reasoner, ["a dax"], [PROPERTY])
reasoner.adapt(adaptation, labels = torch.tensor([1]))

['a dax is a bird.']


In [8]:
properties = ['is an animal', 'is a mammal', 'is a bird', 'is an ape', 'can breathe', 'can grow', 'is a living thing', 'is a plant', 'is a fish', 'can fly']

stimuli = prepare_stimuli_prop(reasoner, ['a dax'], properties)

['a dax is an animal.', 'a dax is a mammal.', 'a dax is a bird.', 'a dax is an ape.', 'a dax can breathe.', 'a dax can grow.', 'a dax is a living thing.', 'a dax is a plant.', 'a dax is a fish.', 'a dax can fly.']


In [9]:
reasoner.generalize(stimuli)

tensor([[-4.4052, -0.0123],
        [-2.5386, -0.0823],
        [-3.4440, -0.0325],
        [-3.1044, -0.0459],
        [-4.6169, -0.0099],
        [-4.8222, -0.0081],
        [-3.4630, -0.0318],
        [-1.9329, -0.1563],
        [-1.0055, -0.4555],
        [-3.3311, -0.0364]])